In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import logging
import pyLSV2
from datetime import datetime, timedelta
import csv
logging.basicConfig(level=logging.INFO)


In [ ]:
#with pyLSV2.LSV2("10.82.0.54", port=19000, timeout=5, safe_mode=False) as con:
with pyLSV2.LSV2("192.168.56.103", port=19000, timeout=5, safe_mode=False) as con:

    availible_signals = con.read_scope_signals()
    # build list with selected signals:
    selected_signals = list()
    selected_signals.append(availible_signals[40])       # channel_number is: 0
    selected_signals.append(availible_signals[41])       # channel_number is: 1
    selected_signals.append(availible_signals[42])       # channel_number is: 2
    selected_signals.append(availible_signals[165])      # channel_number is: 3
    selected_signals.append(availible_signals[166])      # channel_number is: 4
    selected_signals.append(availible_signals[167])      # channel_number is: 5
    selected_signals.append(availible_signals[111])      # channel_number is: 6
    selected_signals.append(availible_signals[112])      # channel_number is: 7
    selected_signals.append(availible_signals[113])      # channel_number is: 8
    selected_signals.append(availible_signals[49])       # channel_number is: 9
    selected_signals.append(availible_signals[174])      # channel_number is: 10
    selected_signals.append(availible_signals[120])      # channel_number is: 11
    selected_signals.append(availible_signals[175])      # channel_number is: 12
    selected_signals.append(availible_signals[176])      # channel_number is: 13
    selected_signals.append(availible_signals[177])      # channel_number is: 14

    duration = 5
    interval = 3000 # in micro second time unit.
    time_counter = 0
    print("selected signals:")
    for sig in selected_signals:
        print("# %s" % sig)

    # take readings:
    with open('data_trial.csv', 'w', newline='') as fp:
        readings_counter = 0
        count_high_freq = 0
        writer = csv.writer(fp)
        field = ["time",
                 "v_actual_X", "v_actual_Y", "v_actual_Z",
                 "a_actual_X", "a_actual_Y", "a_actual_Z",
                 "I_nominal_X", "I_nominal_Y", "I_nominal_Z",
                 "spindle_v_actual", "spindle_a_actual", "spindle_I_nominal",
                 "jerk_actual_X", "jerk_actual_Y", "jerk_actual_Z",
                 "axial_depth", "radial_depth",
                 "tool_diameter", "tool_teeth_num.", "tool_material", "tool_length",
                 "workpiece_material"
                 ]
        writer.writerow(field)

        for package in con.real_time_readings(selected_signals, duration, interval):
            signal_readings = package.get_data()
            readings_per_signal = len(signal_readings[0].data)
            recording_start_time = datetime.now()

            for i in range(readings_per_signal):

                reading_time = recording_start_time + time_counter * timedelta(microseconds=interval)
                #print(reading_time)
                # Signal_type = sample[# appending rank]["data"][one_smaple]
                #for signal in signal_readings:
                #    value = (signal.data[i] * signal.factor) + signal.offset
                #    print(value, signal.unit)

                if count_high_freq % 5 == 0 :
                    # This condition is only for signals of low frequency
                    v_actual_X  = round(signal_readings[0].data[i]  * signal_readings[0].factor ,   3)                    
                    v_actual_Y  = round(signal_readings[1].data[i]  * signal_readings[1].factor ,   3)
                    v_actual_Z  = round(signal_readings[2].data[i]  * signal_readings[2].factor ,   3)
                    a_actual_X  = round(signal_readings[3].data[i]  * signal_readings[3].factor ,   3)
                    a_actual_Y  = round(signal_readings[4].data[i]  * signal_readings[4].factor ,   3)
                    a_actual_Z  = round(signal_readings[5].data[i]  * signal_readings[5].factor ,   3)
                    v_actual_S  = round(signal_readings[9].data[i]  * signal_readings[9].factor ,   3)
                    a_actual_S  = round(signal_readings[10].data[i] * signal_readings[10].factor,   3)
                    r_actual_X  = round(signal_readings[12].data[i] * signal_readings[12].factor,   3)
                    r_actual_Y  = round(signal_readings[13].data[i] * signal_readings[13].factor,   3)
                    r_actual_Z  = round(signal_readings[14].data[i] * signal_readings[14].factor,   3)
                I_nominal_X = round(signal_readings[6].data[i]  * signal_readings[6].factor ,   3)
                I_nominal_Y = round(signal_readings[7].data[i]  * signal_readings[7].factor ,   3)
                I_nominal_Z = round(signal_readings[8].data[i]  * signal_readings[8].factor ,   3)
                I_nominal_S = round(signal_readings[11].data[i] * signal_readings[11].factor,   3)


                data = list()
                data.append(reading_time)
                # From LSV2
                data.append(v_actual_X )
                data.append(v_actual_Y )
                data.append(v_actual_Z )
                data.append(a_actual_X )
                data.append(a_actual_Y )
                data.append(a_actual_Z )
                data.append(I_nominal_X)
                data.append(I_nominal_Y)
                data.append(I_nominal_Z)
                data.append(v_actual_S )
                data.append(a_actual_S )
                data.append(I_nominal_S)
                data.append(r_actual_X)
                data.append(r_actual_Y)
                data.append(r_actual_Z)
                writer.writerow(data)
                count_high_freq += 1

                if time_counter <= 30:
                    time_counter +=1
                else:
                    time_counter = 0
            readings_counter += readings_per_signal

    print("a total of %d samples were taken" % readings_counter)

    print("the signal description was updated to:")
    for s in selected_signals:
        print(s)
